## Project Overview（项目概述）

### 1. 场景与问题背景

本项目面向的是各类**展演空间**，包括但不限于：公共艺术空间、文化中心、社区剧场、图书馆活动区、科技展览馆等。这类场馆通常会持续举办多种类型的公益活动和展演，例如讲座、放映、工作坊、小型演出、主题展览等。

在现实中，这些场馆的活动报名与签到通常存在以下问题：

- **工具分散、数据割裂**  
  很多场馆使用纸质报名表、Excel、零散的在线表单等方式分别收集不同场次的数据。活动结束后，想要查看某一类观众的参与情况或进行长期统计时，需要大量手工汇总和清洗。

- **容量管理混乱，容易“超卖”或浪费名额**  
  对于热门活动或小场地展演，往往需要提前控制报名人数。但在没有统一系统的情况下，手工统计报名人数容易出错：要么超出场地实际容量，要么临时发现有人多次报名、占位不来，导致真正有兴趣的观众报不上名。

- **现场签到效率低、体验不佳**  
  工作人员需要对照纸质名单或临时导出的表格逐一核对姓名，排队时间长、容易漏检或误记，影响场馆形象和观众体验。

- **难以做数据分析和服务优化**  
  场馆往往想知道：不同类型的活动分别吸引了哪些群体？“会员”“媒体”“普通市民”等受众的参与情况如何？不同时间段、不同主题的出勤率有什么规律？但由于数据分散，这些问题很难系统回答。

特别是在**公益场馆**中，一个典型的痛点是：

> 有人提前“占位”报名，却在当天不来入场；而真正有需要 / 有兴趣的观众却因为“名额已满”而无法报名。

这不仅造成资源浪费，也影响场馆对公众服务的公平性。

---

### 2. 系统目标与定位

在上述背景下，本项目希望设计并实现一个面向展演空间的**公益性质活动报名与签到系统**，核心目标包括：

1. **支持多活动、多场次的统一管理**  
   一个活动可以跨多天、划分为多个具体场次（Session），每个场次都有独立的时间与容量控制。

2. **在不涉及收费的前提下，实现严格的容量控制与公平分配**  
   系统**不处理任何票价、支付、退款**等商业票务逻辑，所有活动均为**免费报名**。  
   它更关注的是：在有限座位资源下，用合理的规则（capacity + waiting list + 惩罚机制）让报名过程尽可能公平、有序。

3. **通过 waiting list + no-show 惩罚机制提升资源利用率和公平性**  
   - 当场次报满时，系统启用**候补队列（waiting list）**，按时间顺序排队；一旦有人取消，候补可自动或半自动转为正式名额。  
   - 对于连续多次报名却不来签到的用户（no-show），系统在最近 30 天内累计未签到达到 3 次时，会自动**禁用其 1 个月的报名权限**。  
   - 这两者组合的目的不是“惩罚用户”，而是：
     - 减少“占位不来”导致的名额浪费；
     - 让真正有需要、有意愿到场的观众更有机会获得名额；
     - 从公益资源配置角度提升整体使用效率和公平性。

4. **提供高效的二维码签到流程，简化现场操作**  
   用户报名成功后，系统生成一张包含二维码的“入场凭证”。二维码中以 JSON 的形式记录用户与报名信息。  
   现场工作人员只需通过“展演签到面板”扫描二维码，即可完成签到并显示观众基本信息，从而避免在签到环节出现复杂的并发问题，也显著减少排队核验时间。

5. **支持面向运营方的分析与导出，为后续数据分析提供基础**  
   系统提供数据分析面板，支持统计例如：
   - 今日所有活动/场次的报名总人数；
   - 今日已签到人数；
   - 不同活动类型、不同标签（Tag）下的报名与出勤情况；
   - 按观众群体（如 VIP、媒体、会员、普通市民等 group_name）统计某类活动的参与率。  
   同时提供 CSV 导出，便于场馆进一步在外部工具中进行数据分析与可视化。

---

### 3. 目标用户与角色划分

系统中主要涉及三类角色：

1. **普通观众（Visitor）**
   - 浏览展演空间未来的活动与场次列表；
   - 根据日期、标签、活动类型等条件筛选活动；
   - 对具体场次进行报名和取消报名；
   - 获取包含二维码的入场凭证并在现场扫码签到；
   - 需遵守 no-show 策略：若在最近 30 天内累计 3 次报名但未签到，将被系统自动暂停报名 1 个月。

2. **场馆工作人员（Staff）**
   - 通过后台管理界面，创建/编辑/下线活动与场次；
   - 管理活动标签与观众群体信息；
   - 查询某场活动或某个场次的报名名单和签到记录；
   - 在必要时**强制添加参会人员**（例如 VIP、媒体嘉宾、现场补录等），这类操作不受普通用户层面的 capacity 和 waiting list 限制；
   - 使用签到面板配合二维码为观众快速办理入场。

3. **系统管理员（Admin）**
   - 拥有最高权限的管理账户（通常为唯一或极少数）；
   - 负责管理工作人员账户、全局配置、系统安全与数据一致性；
   - 在需要时可以直接进行底层数据维护（例如应对异常数据、迁移等）。

---

### 4. 核心功能与特点

从功能维度看，系统提供以下几个核心模块：

1. **活动与场次管理（Event & Session Management）**
   - 活动（Event）作为逻辑上的展演单元，可以包含多个场次（Session）。  
   - 每个场次有自己的开始/结束时间、容量（capacity）、候补上限（waiting_list_limit）和状态（开放/关闭）。  
   - 活动有完整的生命周期状态：草稿（draft）、已发布（published）、报名截止（closed）、已归档（archived），用于控制用户端能否看到/报名。

2. **报名与候补机制（Registration & Waiting List）**
   - 当观众对某个场次发起报名时：
     - 若当前已报名人数（current_registered）尚未达到容量（capacity），则直接记为 `registered`；
     - 若已满，则根据 waiting_list_limit 决定是否允许进入 `waiting` 候补队列；
     - 若候补队列也满，则拒绝报名并提示“本场次已满”。
   - 当已注册的观众取消报名时，系统会：
     - 减少该场次的 current_registered；
     - 按队列顺序从 waiting 中挑选观众转为 `registered`，使座位尽可能重新分配给真正有意愿到场的人。

3. **二维码签到与“展演签到面板”（QR Check-in）**
   - 报名成功后，系统为每条报名记录生成一个“电子票”样式的图片，其中嵌入二维码；
   - 二维码内容为 JSON，包括用户 ID、场次 ID、报名记录 ID 等基础信息；
   - 现场工作人员在“展演签到面板”中通过扫码获取信息，系统根据报名记录进行验证并写入签到时间（checkin_time）；
   - 面板即时显示该观众的姓名以及所属群体（group_name）等信息，方便工作人员核验和交流。

4. **爽约统计与报名限制（No-show Tracking & Penalty）**
   - 系统定义“爽约”为：观众对某个场次状态为 `registered`，该场次结束后仍未完成签到；
   - 在观众每次尝试新报名时，系统会统计其最近 30 天内的爽约次数：
     - 若次数达到 3 次，则自动设置该用户的 `blocked_until` 时间为 30 天后；
     - 在 blocked_until 之前，用户无法报名任何新场次；
   - 通过这一机制，系统鼓励观众合理安排时间，减少“占位不来”的行为，提高公益资源的整体利用率。

5. **数据分析与导出（Analytics & Export）**
   - 后台界面提供常用统计：
     - 今日所有活动/场次的报名总人数；
     - 今日所有签到人数；
     - 某一事件类型、标签或时间段下，不同 group_name 群体的参与情况；
   - 支持将某活动或某场次的报名/签到记录导出为 CSV 文件，为后续的统计分析和可视化使用；
   - 在技术实现上，部分分析查询将以“串行 vs 应用层并行（多线程）”两种方式实现，并比较响应时间，以展示并行查询对复杂报表的性能影响。

---

### 5. 技术关注点：并发控制与数据一致性

虽然展演空间的实际并发规模相对温和（例如高峰时约 100 人并发访问），但本项目仍然重点关注：

1. **在并发报名场景下，如何严守容量约束，避免“超卖”**  
   - 利用 InnoDB 的事务与行级锁，在报名流程中对 EVENT_SESSION 的容量字段执行 `SELECT … FOR UPDATE`；
   - 确保同一场次的 current_registered 始终与 REGISTRATION 中状态为 `registered` 的记录数保持一致；
   - 通过并发测试（模拟多个线程同时对同一场次报名）验证在压力下也不会出现超容量问题。

2. **候补队列的正确性与公平性**  
   - 使用 status + 排队字段或注册时间，保证 waiting list 的先来先服务（FIFO）性质；
   - 在取消报名时，自动/半自动转正候补用户，使名额能被更充分地利用。

3. **数据分析查询的性能与可扩展性**  
   - 对常用统计查询合理建立索引；
   - 对后台 dashboard 的多指标统计尝试串行与并行两种执行方式，比较其响应时间。

---

### 6. 开源与可复用性（Open-source & Reusability）

本项目刻意将系统设计为一个**开源、可复用的通用组件**，而不是为某一家特定场馆定制的闭源系统：

- **不绑定任何收费渠道或商业票务平台**  
  系统不处理支付，也不依赖第三方商业票务服务；活动默认是免费参与的。这使其非常适合作为公共机构、公益展演空间的基础设施，而不会与现有的商业售票系统产生冲突。

- **可部署到不同类型的场馆**  
  无论是美术馆、文化中心、社区剧场，还是高校对公众开放的活动空间，都可以在本项目的基础上进行少量定制（例如 UI 主题、用户群体分类）后直接部署使用。

- **为后续研究和扩展预留空间**  
  系统的数据库模型和接口设计尽量清晰、模块化，便于未来在此基础上扩展：
  - 更复杂的权限系统；
  - 推荐算法（根据用户历史报名行为推荐活动）；
  - 与外部统计 / 可视化平台的深度集成等。

通过这样一个开放、可复用的公益活动报名与签到系统，我们希望一方面减轻展演场馆在日常运营中的重复劳动负担，另一方面让有限的公共文化资源能够被更多真正有需要、有兴趣的观众公平地享用。

下面先用中文把你的 Technical Design：软件架构 / 技术栈 / 系统图思路搭好，专门针对：

- 后端：Flask
- 前端：Vue（单页或多页应用）
- 后端是单体应用 + 分层设计

你可以先把这套中文写进草稿，再翻译成英文版本。

---

## 一、总体架构（Overall Architecture）

### 1.1 架构模式

本系统采用**单体 Web 应用 + 前后端分离**的架构模式：

- **前端层（Frontend）**：  
  使用 Vue 构建浏览器端的单页应用（或多页应用），包括：
  - 普通观众的报名浏览界面；
  - 场馆工作人员的后台管理界面；
  - 专用的展演签到面板界面。
  前端通过 RESTful API 与后端通信，所有数据以 JSON 形式交互。

- **后端层（Backend）**：  
  使用 Flask 实现一个**单体后端服务**，内部采用**分层设计**（MVC-ish）：
  - API / Controller 层：定义 REST API 路由，处理 HTTP 请求与响应。
  - Service（业务服务）层：封装核心业务逻辑（报名、候补、惩罚机制、统计等）。
  - Data Access / Repository 层：基于 SQLAlchemy 访问数据库。
  - Database 层：MySQL，负责数据持久化、约束、索引与事务控制。

整体思路可以理解为：**“前端 SPA + 后端 REST API + 关系数据库”** 的典型三层结构，但后端仍然是单体（Monolithic），方便部署与课程演示。

---

## 二、技术栈说明（Technology Stack）

### 2.1 后端（Backend）

- **语言**：Python 3.x
- **Web 框架**：Flask
  - 使用蓝图（Blueprint）划分不同功能模块的 API，如 `events_api`, `registration_api`, `analytics_api` 等。
- **ORM 与数据库访问**：SQLAlchemy
  - 提供 ORM 映射模型（User, Event, EventSession, Registration, Tag 等）。
  - 通过会话（Session）管理事务与查询。
- **数据库**：MySQL 8.x
  - 存储核心业务数据：
    - USER / EVENT / EVENT_SESSION / REGISTRATION / TAG / EVENT_TAG 等表；
  - 使用 InnoDB 引擎，支持行级锁、事务与外键约束；
  - 字符集采用 `utf8mb4`。

- **数据库驱动**：`mysqlclient` 或 `pymysql`（例如使用 `mysql+pymysql` 连接串）。
- **配置管理**：`.env` + `python-dotenv`
  - 从环境变量读取数据库地址、调试开关等配置。
- **并发实验支持**：
  - 使用 Python 标准库的 `concurrent.futures.ThreadPoolExecutor` 模拟高并发报名测试；
  - 对比有 / 无 `SELECT ... FOR UPDATE` 的不同实现。

### 2.2 前端（Frontend）

- **前端框架**：Vue（可使用 Vue 3 + Vite 或 Vue CLI）
- **UI 技术**：
  - Vue 组件化构建页面；
  - 使用 Axios 或 Fetch 调用后端 REST API；
  - 基础样式可用 Element Plus / Ant Design Vue 等组件库，也可以手写简单 CSS。
- **前端路由对象**（示例）：
  - `/`：普通观众首页（活动列表 + 筛选）
  - `/events/:id`：活动详情 + 场次列表
  - `/my-registrations`：我的报名列表
  - `/admin/events`：后台活动管理
  - `/admin/analytics`：数据分析面板
  - `/checkin`：展演签到面板（扫码界面）

前后端通过统一的 JSON 格式进行数据交互，前端不直接访问数据库。

### 2.3 其他工具（DevOps / 辅助）

- **接口测试**：Postman / curl / VSCode REST Client
- **图表与文档**：
  - ER 图 / 系统结构图：draw.io / Mermaid / Graphviz
- **测试**：
  - Python `unittest` 或 `pytest` 用于核心业务逻辑和并发测试脚本。

---

## 三、后端分层设计与模块职责

为了让技术设计更清晰，你可以把 Flask 后端划分为几个逻辑层：

### 3.1 API / Controller 层

- 形式：Flask Blueprint + 路由函数。
- 职责：
  - 接收 HTTP 请求（GET/POST/PUT/DELETE 等）；
  - 解析路径参数、查询参数、请求体；
  - 进行基础校验（例如参数是否为空、类型是否正确）；
  - 调用对应的 Service 层函数执行业务操作；
  - 将结果封装为 JSON 响应返回给前端；
  - 处理常见错误（如认证失败、权限不足、参数错误），返回合适的 HTTP 状态码。

- 示例模块：
  - `events_api.py`：活动与场次的相关接口（列表、详情、标签筛选等）；
  - `registration_api.py`：报名、取消报名、查询用户报名记录等；
  - `checkin_api.py`：通过二维码完成签到；
  - `analytics_api.py`：后台统计与导出；
  - `auth_api.py`：登录 / 用户信息（如需要简单认证）。

### 3.2 Service（业务逻辑）层

- 形式：独立的 Python 模块 / 类。
- 职责：
  - 实现真正的业务逻辑与规则，而不是嵌在路由函数里；
  - 例如：
    - 报名流程：检查用户 blocked 状态 → 检查活动/场次状态 → 加锁读取 capacity → 决定 registered / waiting → 更新 current_registered；
    - 取消报名流程 + waiting list 自动转正；
    - 统计某用户过去 30 天 no-show 次数，如达到 3 次则更新 blocked_until；
    - 生成数据分析结果（聚合查询、分组统计等）。

- 典型 Service：
  - `RegistrationService`：handle_register, handle_cancel, compute_no_show_and_block；
  - `EventService`：list_events, list_sessions, manage tags, check event/session status；
  - `AnalyticsService`：统计每日报名 / 签到、按 group_name 分布、导出 CSV 等；
  - `UserService`：管理用户信息、角色、group_name 等。

### 3.3 Data Access / Repository 层

- 形式：SQLAlchemy ORM 模型 + Repository 函数。
- 职责：
  - 提供对数据库的封装，以便 Service 层更容易读写数据；
  - 例如：
    - `get_session_by_id(session_id)`；
    - `count_waiting_users(session_id)`；
    - `get_no_shows_in_last_30_days(user_id)`；
    - `get_daily_registration_stats(date)`；
  - 保证所有数据库操作都在统一的会话管理（SessionLocal）下进行。

### 3.4 Database 层（MySQL）

- 职责：
  - 存储持久化数据（USER / EVENT / EVENT_SESSION / REGISTRATION / TAG 等）；
  - 通过主键、外键、UNIQUE 和 CHECK 约束保证数据完整性；
  - 通过索引提升查询效率；
  - 支持事务和行级锁，为高并发报名下的容量控制提供支持。

---

## 四、示例后端目录结构

你可以在报告里展示一个类似这样的目录结构，体现“架构设计”而不是只有一个 app.py：

```text
backend/
  app.py                # Flask 应用入口，创建 app，注册蓝图
  config.py             # 配置加载（读取 .env 等）
  db/
    __init__.py
    db.py               # create_engine, SessionLocal, get_db()
    schema.sql          # DDL：表、索引、约束
  models/
    __init__.py
    user.py
    event.py
    session.py
    registration.py
    tag.py
  repositories/
    __init__.py
    user_repo.py
    event_repo.py
    session_repo.py
    registration_repo.py
    analytics_repo.py
  services/
    __init__.py
    user_service.py
    event_service.py
    registration_service.py
    analytics_service.py
  api/
    __init__.py
    auth_api.py
    events_api.py
    registration_api.py
    analytics_api.py
    checkin_api.py
  utils/
    qr_code.py          # 生成二维码图片，封装 JSON payload
    security.py         # 简单 token 签名/验证（可选）
tests/
  test_registration.py  # 报名与候补的单元测试
  test_concurrency.py   # 并发报名压力测试
```

这样你就可以在 Technical Design 章节中清晰说明：

- 哪一层做什么；
- 路由/API 与业务逻辑解耦；
- 数据访问集中管理。

---

## 五、高层系统图（System Diagram）思路

你需要一张**系统结构图**，表达前后端和数据库之间的关系。可以用 Mermaid 或 draw.io 画。结构大致如下：

### 5.1 概念说明

**参与方：**

- Visitor Browser（普通观众浏览器）
- Staff/Admin Browser（后台管理界面）
- Check-in Panel Browser（签到面板浏览器）

**系统组件：**

- Vue Frontend（前端应用）
  - 由多个 Vue 组件页面构成，调用后端 REST API。
- Flask Backend（单体后端）
  - API/Controller 层
  - Service 层
  - Repository / ORM 层
- MySQL Database

**数据流：**

- 浏览器 → 前端 Vue：用户操作（点击/输入）
- 前端 Vue → 后端 Flask：通过 Axios 发起 HTTP 请求（JSON）
- Flask Backend → MySQL：通过 SQLAlchemy 读写数据
- MySQL → Flask → Vue → 浏览器：返回数据并渲染界面

### 5.2 系统图

![SystemDiagram](SysytemDiagram.png)

下面是一版按“作业报告结构”整理好的**完整中文版 Database Design（Track 1）**，已经包含你最新的需求修改（group 变成按活动配置、默认 Member、由管理员维护），方便你之后直接翻成英文。

---

# 一、ER 设计（ER Diagram）

## 1.1 核心实体与属性

### 1. USER

- 含义：系统中的用户，包括观众（visitor）、工作人员（staff）、管理员（admin）。
- 主要属性：
  - **user_id**：主键，标识用户。
  - **name**：姓名。
  - **email**：登录邮箱，唯一。
  - **password**：密码（加密存储）。
  - **role**：用户角色，取值范围：visitor / staff / admin。
  - **blocked_until**：禁用报名的截止时间（DATETIME，NULL 表示未被禁用）。

> 调整点：**不再**在 USER 表上保存 group_name，观众分组由“用户 + 活动”层级单独管理。

---

### 2. ORGANIZATION（可选）

- 含义：活动的主办方或承办机构。
- 主要属性：
  - **org_id**：主键。
  - **org_name**：机构名称，唯一。
  - **contact_email**：联系邮箱（可空）。

---

### 3. EVENTTYPE

- 含义：活动类型，用于对 EVENT 做分类。
- 主要属性：
  - **type_id**：主键。
  - **type_name**：类型名称，唯一，例如 Exhibition / Concert / Talk / Workshop 等。

---

### 4. EVENT

- 含义：一个活动项目的“整体”，例如“2025 春季艺术展”。
- 特点：一个 EVENT 可以包含多场 EVENT_SESSION。
- 主要属性：
  - **eid**：主键。
  - **org_id**：外键，指向 ORGANIZATION(org_id)，可空。
  - **type_id**：外键，指向 EVENTTYPE(type_id)，不可空。
  - **title**：活动标题。
  - **description**：活动描述（可空）。
  - **location**：活动地点。
  - **status**：活动状态，取值：
    - draft / published / closed / archived。
  - **created_at**：创建时间。
  - **updated_at**：更新时间。

---

### 5. EVENT_SESSION

- 含义：某个活动下的具体场次（Session），如“12 月 20 日 10:00–12:00”。
- 特点：一个 EVENT 对应多个 EVENT_SESSION。
- 主要属性：
  - **session_id**：主键。
  - **eid**：外键，指向 EVENT(eid)，不可空。
  - **start_time**：开始时间。
  - **end_time**：结束时间。
  - **capacity**：场次容量（>0）。
  - **current_registered**：当前已成功报名人数（≥0）。
  - **waiting_list_limit**：候补名单上限（≥0）。
  - **status**：场次状态，取值：open / closed。

---

### 6. TAG

- 含义：活动标签，例如 Art、Music、Talk、Workshop 等。
- 主要属性：
  - **tag_id**：主键。
  - **tag_name**：唯一标签名。

---

### 7. EVENT_TAG

- 含义：EVENT 与 TAG 的多对多关系表。
- 主要属性：
  - **eid**：外键，指向 EVENT(eid)。
  - **tag_id**：外键，指向 TAG(tag_id)。
- 主键：
  - 复合主键 (eid, tag_id)。

---

### 8. REGISTRATION

- 含义：用户对某一场次的报名记录。
- 主要属性：
  - **user_id**：外键，指向 USER(user_id)。
  - **session_id**：外键，指向 EVENT_SESSION(session_id)。
  - **register_time**：报名时间。
  - **status**：报名状态，取值：
    - registered（已成功报名）
    - waiting（候补中）
    - cancelled（已取消）
  - **checkin_time**：签到时间（NULL 表示未签到）。
  - **queue_position**：候补队列中的排序位置（可空，仅对 waiting 有意义）。
- 主键：
  - 复合主键 (user_id, session_id)，保证同一用户对同一场次只有一条记录。

---

### 9. AUDIENCE_GROUP（新增：观众群体配置）

- 含义：系统中可配置的“观众群体类型”，由管理员维护。
- 示例：Member（默认）、VIP、Media、Student 等。
- 主要属性：
  - **group_id**：主键。
  - **group_name**：唯一名称（如 'Member', 'VIP'）。
  - **description**：说明（可空）。
  - **is_default**：是否为系统默认分组（布尔型），例如 Member = TRUE。

> 系统层面只需要保证至少存在一个默认分组（例如 Member），剩下的分组完全由管理员在后台添加和维护。

---

### 10. EVENT_USER_GROUP（新增：用户在某活动下的分组）

- 含义：描述“某用户在某活动中属于哪个观众分组”。
- 主要属性：
  - **user_id**：外键，指向 USER(user_id)。
  - **eid**：外键，指向 EVENT(eid)。
  - **group_id**：外键，指向 AUDIENCE_GROUP(group_id)。
- 主键：
  - PRIMARY KEY (user_id, eid)  
    表示一个用户在同一个活动中只属于一个观众群体。

> 业务规则：  
> - 用户第一次报名某活动的任意场次时，如果没有特别指定分组，就自动分配到默认 group（AUDIENCE_GROUP 中 is_default = TRUE 的那条记录，例如 Member）。

---

## 1.2 主要实体关系

- **USER – REGISTRATION：1 对多**
  - 一个用户可以有多条报名记录；
  - 每条报名记录只属于一个用户。

- **EVENT – EVENT_SESSION：1 对多**
  - 一个活动可以有多个不同时间的场次；
  - 每个场次只隶属于一个活动。

- **EVENT_SESSION – REGISTRATION：1 对多**
  - 一个场次可以被多个用户报名；
  - 每条报名记录只对应一个场次。

- **EVENT – TAG：多对多（通过 EVENT_TAG 实现）**
  - 一个活动可以有多个标签；
  - 一个标签也可以作用于多个活动。

- **EVENT – EVENTTYPE：多对一**
  - 一个活动有一个类型；
  - 一个类型可以对应多个活动。

- **EVENT – ORGANIZATION：多对一**
  - 一个活动由某个机构主办；
  - 一个机构可以主办多个活动。

- **AUDIENCE_GROUP – EVENT_USER_GROUP：1 对多**
  - 一个观众群体类别可以被很多用户在不同活动中使用；
  - 每条 EVENT_USER_GROUP 只对应一个 group_id。

- **USER – EVENT – AUDIENCE_GROUP：三元关系（由 EVENT_USER_GROUP 实现）**
  - EVENT_USER_GROUP(user_id, eid, group_id) 表示：  
    “在某个活动中，该用户属于某个观众群体”。

---

## 1.3 ER 图（Mermaid 代码，仅示意）
![ERdiag1](ERdiag1.png)


---

# 二、Schema 设计（表结构 / 主外键 / 约束）

下面按表逐个描述字段、主键、外键、约束。

## 2.1 USER 表

- 字段：
  - user_id INT PK AUTO_INCREMENT
  - name VARCHAR
  - email VARCHAR UNIQUE NOT NULL
  - password VARCHAR NOT NULL
  - role ENUM('visitor','staff','admin') NOT NULL
  - blocked_until DATETIME NULL
- 约束：
  - email 必须唯一；
  - role 只能取 visitor / staff / admin；
  - blocked_until 允许为 NULL（未被禁用）。

---

## 2.2 ORGANIZATION 表（可选）

- 字段：
  - org_id INT PK AUTO_INCREMENT
  - org_name VARCHAR UNIQUE NOT NULL
  - contact_email VARCHAR NULL

---

## 2.3 EVENTTYPE 表

- 字段：
  - type_id INT PK AUTO_INCREMENT
  - type_name VARCHAR UNIQUE NOT NULL

---

## 2.4 EVENT 表

- 字段：
  - eid INT PK AUTO_INCREMENT
  - org_id INT NULL FK → ORGANIZATION(org_id)
  - type_id INT NOT NULL FK → EVENTTYPE(type_id)
  - title VARCHAR NOT NULL
  - description TEXT NULL
  - location VARCHAR NOT NULL
  - status ENUM('draft','published','closed','archived') NOT NULL
  - created_at DATETIME NOT NULL
  - updated_at DATETIME NOT NULL

---

## 2.5 EVENT_SESSION 表

- 字段：
  - session_id INT PK AUTO_INCREMENT
  - eid INT NOT NULL FK → EVENT(eid)
  - start_time DATETIME NOT NULL
  - end_time DATETIME NOT NULL
  - capacity INT NOT NULL CHECK (capacity > 0)
  - current_registered INT NOT NULL DEFAULT 0 CHECK (current_registered >= 0)
  - waiting_list_limit INT NOT NULL DEFAULT 0 CHECK (waiting_list_limit >= 0)
  - status ENUM('open','closed') NOT NULL
- 逻辑约束（应用层确保）：
  - start_time < end_time；
  - current_registered ≤ capacity。

---

## 2.6 TAG 表

- 字段：
  - tag_id INT PK AUTO_INCREMENT
  - tag_name VARCHAR UNIQUE NOT NULL

---

## 2.7 EVENT_TAG 表

- 字段：
  - eid INT NOT NULL FK → EVENT(eid)
  - tag_id INT NOT NULL FK → TAG(tag_id)
- 主键：
  - PRIMARY KEY (eid, tag_id)

---

## 2.8 REGISTRATION 表

- 字段：
  - user_id INT NOT NULL FK → USER(user_id)
  - session_id INT NOT NULL FK → EVENT_SESSION(session_id)
  - register_time DATETIME NOT NULL
  - status ENUM('registered','waiting','cancelled') NOT NULL
  - checkin_time DATETIME NULL
  - queue_position INT NULL
- 主键：
  - PRIMARY KEY (user_id, session_id)
- 业务语义约束：
  - status = 'registered'：计入 EVENT_SESSION.current_registered；
  - status = 'waiting'：进入候补队列，可按 queue_position 排序；
  - status = 'cancelled'：不再计入人数；
  - checkin_time 非 NULL：表示已签到；
  - no-show：场次结束后仍为 status='registered' 且 checkin_time IS NULL。

---

## 2.9 AUDIENCE_GROUP 表（观众群体配置）

- 字段：
  - group_id INT PK AUTO_INCREMENT
  - group_name VARCHAR UNIQUE NOT NULL
  - description VARCHAR NULL
  - is_default BOOLEAN NOT NULL DEFAULT FALSE
- 约束与逻辑：
  - group_name 唯一；
  - 应保证系统中至少存在一个 is_default = TRUE（例如 'Member'）；
  - 管理员可在后台新增/修改/删除观众群体配置。

---

## 2.10 EVENT_USER_GROUP 表（用户在活动中的分组）

- 字段：
  - user_id INT NOT NULL FK → USER(user_id)
  - eid INT NOT NULL FK → EVENT(eid)
  - group_id INT NOT NULL FK → AUDIENCE_GROUP(group_id)
- 主键：
  - PRIMARY KEY (user_id, eid)
- 语义：
  - 同一用户在同一活动中只对应一个观众群体；
  - 用户首次参与某活动时，如果无特别指定，系统自动插入一条记录，将其分配到默认分组（例如 Member）。

---

# 三、规范化程度（Normalization Level）

本设计至少满足第三范式（3NF），接近 BCNF。

## 3.1 第一范式（1NF）

- 所有表的每个字段都是原子值，没有数组/重复列；
- 例如：
  - TAG 以行记录形式存储，每个 tag_name 独立一条记录；
  - EVENT_TAG 用关联表表示多对多，而不是在 EVENT 中用字符串保存多个标签；
  - EVENT_USER_GROUP 用独立记录表示用户在某活动下的分组，而不是把多个 group 拼在一个字段里。

---

## 3.2 第二范式（2NF）

- 对于使用单一主键的表（USER, EVENT, EVENT_SESSION, TAG, ORGANIZATION, EVENTTYPE, AUDIENCE_GROUP）：
  - 所有非主键属性都完全依赖主键，不存在“部分依赖”。
- 对于使用复合主键的表：
  - REGISTRATION(user_id, session_id)：  
    register_time, status, checkin_time, queue_position 等，都依赖于 (user_id, session_id) 这一组合，而不是仅依赖其中一个字段。
  - EVENT_TAG(eid, tag_id)：  
    仅作为关联关系，不存在其他非主属性。
  - EVENT_USER_GROUP(user_id, eid)：  
    group_id 依赖于 (user_id, eid) 组合键。

---

## 3.3 第三范式（3NF）

- 消除非主属性之间的传递依赖：
  - EVENTTYPE、ORGANIZATION、TAG、AUDIENCE_GROUP 等拆成独立表，通过 id 在其他表引用，避免在 EVENT 或 REGISTRATION 中重复保存 type_name / org_name / group_name；
  - 用户的角色（role）是 USER 自身属性，不重复冗余在 REGISTRATION 或 EVENT 中；
  - 观众群体定义（group_name）不放在 USER 或 REGISTRATION 中，而由 AUDIENCE_GROUP + EVENT_USER_GROUP 组合管理，避免重复与不一致。
- 因此，核心业务表（USER, EVENT, EVENT_SESSION, REGISTRATION, TAG, EVENT_TAG, AUDIENCE_GROUP, EVENT_USER_GROUP）满足 3NF，减少了冗余和更新异常。

---

# 四、索引策略（Indexing Strategy）

从两个角度说明：在线事务（OLTP）查询以及统计分析（Analytics）。

## 4.1 主键索引

所有主键自动建立索引，例如：

- USER(user_id)
- ORGANIZATION(org_id)
- EVENTTYPE(type_id)
- EVENT(eid)
- EVENT_SESSION(session_id)
- TAG(tag_id)
- AUDIENCE_GROUP(group_id)
- EVENT_TAG(eid, tag_id)
- REGISTRATION(user_id, session_id)
- EVENT_USER_GROUP(user_id, eid)

---

## 4.2 常用业务访问路径索引

### 4.2.1 USER 表

- UNIQUE(email)：
  - 支持登录验证和防止重复注册。
- INDEX(role)：
  - 支持后台按角色过滤用户（如列出所有 staff / admin）。
- INDEX(blocked_until)：
  - 在报名流程中快速判断用户是否处于禁用期。

---

### 4.2.2 EVENT 表

- INDEX(type_id, status)：
  - 按活动类型与状态筛选活动（例如只显示已发布的某一类活动）。
- INDEX(created_at) 或 INDEX(updated_at)：
  - 支持按时间排序活动列表和简单时间统计。

---

### 4.2.3 EVENT_SESSION 表

- INDEX(eid)：
  - 快速查询某活动的全部场次。
- INDEX(start_time) / INDEX(end_time)：
  - 便于按时间范围查询，如“今天的场次”、“未来一周的场次”。
- INDEX(status)：
  - 快速过滤出开放报名的场次（status='open'）。

---

### 4.2.4 REGISTRATION 表

- 主键已为 (user_id, session_id)。
- INDEX(user_id)：
  - 查询某用户的历史报名记录（个人中心/用户管理）。
- INDEX(session_id)：
  - 查询某场次的报名列表（签到、控制入场）。
- 组合索引：
  - INDEX(session_id, status)：  
    - 查询某场次所有 registered / waiting 用户时使用。
  - INDEX(user_id, status)（可选）：  
    - 支持按用户 + 状态统计，例如统计某用户过去的 no-show 次数。
- INDEX(checkin_time)：
  - 按签到时间做统计，例如“今天签到人数”。

---

### 4.2.5 EVENT_TAG 表

- 除了主键 (eid, tag_id) 外：
  - INDEX(tag_id)：  
    - 按标签查找相关活动时使用。

---

### 4.2.6 AUDIENCE_GROUP / EVENT_USER_GROUP 表

- AUDIENCE_GROUP：
  - UNIQUE(group_name)；
  - INDEX(is_default)：  
    - 找到默认分组（比如 Member）时使用。
- EVENT_USER_GROUP：
  - PK(user_id, eid)；
  - INDEX(eid, group_id)：  
    - 按活动与观众群体统计报名情况时使用。
  - INDEX(user_id)：  
    - 查询某用户在各活动中的分组（后台用户分析时）。

---

## 4.3 Analytics 查询优化说明

- 按日期统计签到人数：
  - 依赖 REGISTRATION.checkin_time 的范围查询，建立索引可以减少全表扫描。
- 按观众群体与活动类型统计参与情况：
  - 需要 JOIN USER, REGISTRATION, EVENT_SESSION, EVENT, EVENTTYPE, EVENT_USER_GROUP, AUDIENCE_GROUP；
  - 在 EVENT.type_id、AUDIENCE_GROUP.group_name、EVENT_USER_GROUP.group_id、REGISTRATION.status 等字段上适当建立索引，可以显著提升统计查询性能。
- 设计上在 **3NF 的规范化** 基础上，通过**索引**平衡读写性能与统计需求。

---

# 五、示例查询（Sample Queries）

以下示例展示业务查询、惩罚/no-show 相关逻辑，以及按观众群体/标签的统计查询。

## 5.1 列出某一天内所有已发布且开放的场次及已报名人数

查询条件：  
- 活动状态为 published；
- 场次状态为 open；
- start_time 在指定日期（示例中用当天）。

````markdown
```sql
SELECT
    e.eid,
    e.title,
    s.session_id,
    s.start_time,
    s.end_time,
    s.capacity,
    s.current_registered
FROM EVENT e
JOIN EVENT_SESSION s ON e.eid = s.eid
WHERE e.status = 'published'
  AND s.status = 'open'
  AND DATE(s.start_time) = CURDATE();
```
````

---

## 5.2 查询用户近 30 天内的 no-show 次数

定义 no-show：  
- 对应场次已结束；
- REGISTRATION.status = 'registered'；
- checkin_time IS NULL。

````markdown
```sql
SELECT COUNT(*) AS no_show_count
FROM REGISTRATION r
JOIN EVENT_SESSION s ON r.session_id = s.session_id
WHERE r.user_id = :uid
  AND r.status = 'registered'
  AND r.checkin_time IS NULL
  AND s.end_time >= (NOW() - INTERVAL 30 DAY)
  AND s.end_time < NOW();
```
````

> 后端可根据 no_show_count 来更新 USER.blocked_until，实现惩罚机制。

---

## 5.3 按标签搜索已发布活动

````markdown
```sql
SELECT DISTINCT e.*
FROM EVENT e
JOIN EVENT_TAG et ON e.eid = et.eid
JOIN TAG t ON et.tag_id = t.tag_id
WHERE e.status = 'published'
  AND t.tag_name = :tagName
ORDER BY e.created_at DESC;
```
````

---

## 5.4 统计今天所有场次的报名总人数（registered）

````markdown
```sql
SELECT
    DATE(s.start_time) AS day,
    COUNT(*) AS total_registrations
FROM REGISTRATION r
JOIN EVENT_SESSION s ON r.session_id = s.session_id
WHERE r.status = 'registered'
  AND DATE(s.start_time) = CURDATE()
GROUP BY DATE(s.start_time);
```
````

---

## 5.5 统计今天所有场次的签到人数

````markdown
```sql
SELECT
    DATE(r.checkin_time) AS day,
    COUNT(*) AS total_checkins
FROM REGISTRATION r
WHERE r.checkin_time IS NOT NULL
  AND DATE(r.checkin_time) = CURDATE()
GROUP BY DATE(r.checkin_time);
```
````

---

## 5.6 按观众群体和活动类型统计参与人数（使用 AUDIENCE_GROUP）

例如：统计某一活动类型下，不同观众群体的报名人数（只算 registered 状态）。

````markdown
```sql
SELECT
    g.group_name,
    COUNT(*) AS registrations
FROM REGISTRATION r
JOIN EVENT_SESSION s ON r.session_id = s.session_id
JOIN EVENT e ON s.eid = e.eid
JOIN EVENTTYPE et ON e.type_id = et.type_id
JOIN EVENT_USER_GROUP ug
    ON ug.user_id = r.user_id
   AND ug.eid = e.eid
JOIN AUDIENCE_GROUP g ON ug.group_id = g.group_id
WHERE r.status = 'registered'
  AND et.type_name = :typeName
GROUP BY g.group_name
ORDER BY registrations DESC;
```
````

> 如果需要只统计已签到人数，可以在 WHERE 条件中再加：
> - AND r.checkin_time IS NOT NULL。

# 开发记忆

现在这条链路你已经有了：

- 活动/场次查询：`/api/events/...`
- 报名：`POST /api/registrations/`

要做一个“像样”的 Event Management demo，接下来按优先级可以这样推进，我给你一个循序渐进的 roadmap，你可以选一条我帮你细化代码。

---

## 优先级 1：把报名闭环做完整

### 1. 取消报名接口（必做）

场景：用户想退选 / 取消报名，释放名额 + 推进候补队列。

- 路由建议：
  - `POST /api/registrations/cancel`
- 请求：
  - body: `{ "user_id": 1, "session_id": 1 }`
- 逻辑（核心要点）：
  1. 在事务中锁定 REGISTRATION 记录 + 场次记录
  2. 如果当前状态是：
     - \['registered'\]：
       - 将该记录改为 \['cancelled'\]
       - EVENT_SESSION.current_registered - 1
       - 如果存在 waiting 列表中第一个人：
         - 把该 waiting 记录改为 \['registered'\]，queue_position 置空
         - EVENT_SESSION.current_registered + 1
     - \['waiting'\]：
       - 将其改为 \['cancelled'\]
       - 其他 waiting 的 queue_position 需要 -1（保持连续）
  3. 返回当前用户的新状态 + 是否有人从候补转正

如果你愿意，我可以直接给你：

- `backend/services/registration_service.py` 里新增 `cancel_registration(...)`
- `backend/api/registration_api.py` 里新增 `POST /cancel` 的实现

---

### 2. 查询“我的报名”（实用）

场景：前端“我的活动”页面。

- 路由建议：
  - `GET /api/registrations/user/<user_id>`
- 返回内容：
  - 用户报名过的所有 session，包括：活动标题、场次时间、状态、队列序号等
- SQL 大致：

```sql
SELECT
  r.session_id,
  r.status,
  r.queue_position,
  r.register_time,
  s.start_time,
  s.end_time,
  e.eid,
  e.title,
  e.location
FROM REGISTRATION r
JOIN EVENT_SESSION s ON r.session_id = s.session_id
JOIN EVENT e ON s.eid = e.eid
WHERE r.user_id = %s
ORDER BY s.start_time ASC;
```

这个接口实现起来很快，我也可以直接给你 `GET /api/registrations/user/<int:user_id>` 的完整代码。

---

### 3. check-in 接口（可选但好看）

场景：现场扫码签到，标记用户为“已到场”。

- 路由：
  - `POST /api/registrations/checkin`
- Body：
  - `{ "user_id": 1, "session_id": 1 }`
- 逻辑：
  - 找到 registration 记录（status 必须是 \['registered'\]）
  - 更新 `checkin_time = NOW()`
  - 返回当前记录

---

## 优先级 2：活动管理端（后台功能）

如果你想做“管理员视角”的功能，可以增加：

### 4. 创建 / 编辑活动 + 场次（CRUD）

- `POST /api/events/`
  - 创建 EVENT + 多个 EVENT_SESSION（一次提交）
- `PUT /api/events/<eid>`
  - 更新事件基本信息、状态
- `POST /api/events/<eid>/sessions`
  - 为某个 event 新增一个 session
- `PUT /api/events/sessions/<session_id>`
  - 修改场次时间 / 容量 / 状态（open/closed）

这些比较多，我建议先搞一个最小版：只做“创建一个新的 EVENT + 一个 SESSION”。

---

## 优先级 3：标签 & 分组（锦上添花）

- `GET /api/events/<eid>/detail`
  - 返回：活动基本信息 + 所有场次 + 标签列表 array
- `GET /api/tags/`：
  - 标签列表（来自 TAG 表）
- `POST /api/events/<eid>/tags`
  - 管理员为活动绑定标签

---

## 我建议你现在先做的事

为了尽快让“报名流程”完整起来，我建议顺序：

1. **取消报名**：`POST /api/registrations/cancel`
2. **我的报名列表**：`GET /api/registrations/user/<user_id>`

你告诉我：

- 先做 [1] 取消报名，还是
- 先做 [2] “我的报名”列表，或者
- 两个都要，我按顺序给你完整代码

比如你回复：“先做取消报名”，我就直接给你：

- `registration_service.py` 里新增 `cancel_registration()`
- `registration_api.py` 里新增对应路由
- 加上 curl 测试例子，帮你把业务规则写清楚。

## 后端 API 速查（含角色/示例 JSON）

- **Auth**
  - `POST /api/auth/login` → {email,password} → 返回 {user, token, expires_at}
  - `GET /api/auth/me` → 需 Authorization: Bearer <token>
- **Events & Tags**（staff/admin 才能创建/改标签）
  - `POST /api/events/` {title, description?, location, status, type_id?, sessions:[{start_time,end_time,capacity,waiting_list_limit?}]} → 201
  - `GET /api/events/` 已发布活动列表；`GET /api/events/<eid>/sessions` 场次列表；`GET /api/events/<eid>/detail` 活动+场次
  - `GET /api/events/tags` 列表；`POST /api/events/tags` {tag_name}
  - `POST /api/events/<eid>/tags` {tag_names:["Art","Workshop"]} 覆盖式设置
- **搜索（ORM 多表联查）**
  - `GET /api/events/search?tag=Art&tag=Music&type_id=1&status=published&start=2025-12-01&end=2025-12-31&q=Hall&limit=50&offset=0`
  - 返回 Event 列表，包含 sessions 与 tags
- **Registration / QR / Check-in**
  - `POST /api/registrations/` {session_id}（需登录，含封禁检查/候补逻辑）
  - `POST /api/registrations/cancel` {session_id} 取消并自动候补转正
  - `GET /api/registrations/me` 当前用户报名列表
  - `GET /api/registrations/qrcode/<session_id>` 当前用户该场次二维码 PNG
  - `POST /api/checkin/` {user_id, session_id} 标记签到
- **观众群组（非权限组）**
  - `GET /api/groups` 列出；`POST /api/groups` {group_name, description?, is_default?}（staff/admin）
  - `POST /api/events/<eid>/users/<user_id>/group` {group_id} 分配/更新群组（staff/admin）
  - `GET /api/events/<eid>/users/<user_id>/group`
- **Analytics（staff/admin）**
  - `GET /api/analytics/events/<eid>/overview` 活动概览；`GET /api/analytics/sessions/<id>` 场次统计；`GET /api/analytics/users/me` 我的统计
  - `GET /api/analytics/events/<eid>/trend?start=2025-12-01&end=2025-12-31` 日报名趋势
  - `GET /api/analytics/events/<eid>/group-stats` 按观众群体统计 {group_name, registrations, checkins}
  - `GET /api/analytics/tags/<tag_name>/overview` 标签下活动的 {session_count, registrations, checkins}

> 角色约定：登录后 `g.current_user.role` 可取 visitor | staff | admin。staff/admin 可进行：创建活动/标签、设置活动标签、分配观众群组、运行扩展统计。


## 测试策略与结果（Testing Summary）

- **测试框架**：pytest（含 markers：`requires_db` 真连测试、`real_auth` 使用真实 JWT、`perf` 性能基准）。
- **运行方式**：
  - 仅单元/打桩测试：`pytest -q tests -m "not requires_db"`
  - 包含真连 + 并发 + 性能：确保 `.env` 中 MySQL 可连，然后 `pytest -q tests`
- **数据库准备**：`db_ready` fixture 会自动调用 `init_db_from_schema` + `seed_example_data` 中的种子函数；不可连时自动跳过相关用例。
- **最新结果**：17 passed, 1 warning（SA DISTINCT ON 在非 PostgreSQL 后端被忽略，未来将改为兼容写法）。

| 模块 | 文件 | 说明 | Marker |
| --- | --- | --- | --- |
| Smoke | `test_app_smoke.py` | 根路径与 `/api/events/health` 存活检查 | - |
| Search 单元 | `test_search_service_unit.py` | `_parse_dt` 解析、异常包装 | - |
| Events API | `test_events_api_integration.py` | 搜索接口打桩、创建鉴权校验 | - |
| 真连基础 | `test_db_integration.py` | 登录获取 JWT、service 级报名/取消、活动详情 | `requires_db`, `real_auth` 部分 |
| 全量 API | `test_api_full.py` | 注册/取消、群组 CRUD、analytics 概览（真实 JWT） | `requires_db`, `real_auth` |
| 并发报名 | `test_concurrency_registration.py` | 500 人抢 25 名额，确认不超卖、候补人数一致 | `requires_db`, `perf` |
| 性能基准 | `test_query_performance.py` | `search_events` / `get_event_overview` < 1s | `requires_db`, `perf` |

- **并发测试细节**：使用 `ThreadPoolExecutor(max_workers=50)` 模拟 500 次报名；断言 `registered==25`、`waiting==475`。
- **性能测试阈值**：当前示例数据集下 <1s；如数据量增长，可调整阈值或改为采样/指标上报。
- **SA 警告说明**：`search_events` 使用 DISTINCT ON，MySQL 会忽略，未来 SQLAlchemy 可能抛 CompileError；后续可改为子查询去重方案。
- **后续可扩展**：
  1) 增加导出/标签/群组的端到端用例（含 CSV 下载校验）。
  2) 针对 analytics 复杂查询添加更大规模数据集并记录耗时曲线。
  3) 在 CI 中分阶段执行：基础用例默认跑；`requires_db`/`perf` 由专门任务跑。

## 可用性测试用例表（Usability Test Matrix）

| Scenario | Purpose | Steps | Metric / Check | Expected Result | Executor | Execution Result |
| --- | --- | --- | --- | --- | --- | --- |
| Browse published events | 确认访客能查看活动列表并按标签过滤 | 1) 访问 `/` 2) 点击标签筛选 `Art` | 列表渲染时间 < 1s；过滤后仅显示匹配标签 | 活动列表可见且筛选正确，无空白或错误提示 | 夏晋 | 全通过 |
| Register for an open session | 验证基本报名流程 | 1) 登录访客 2) 打开某 open 场次 3) 点击报名 | 请求 200；UI 显示状态 `registered` | 名额减少，用户状态为 registered，弹出成功提示 | 夏晋 | 全通过 |
| Waitlist when full | 确认满额时进入候补 | 1) 选择容量已满场次 2) 点击报名 | 请求 200；UI 显示状态 `waiting`；队列号展示 | current_registered 不变，用户进入 waiting，显示队列号 | 夏晋 | 全通过 |
| Cancel to free seat | 验证取消后候补转正 | 1) 已 registered 用户取消 2) 候补用户刷新 | 取消请求 200；候补用户状态切换 | 取消者变为 cancelled，候补首位变 registered，席位数保持正确 | 夏晋 | 全通过 |
| QR check-in | 验证现场签到体验 | 1) 打开签到面板 2) 扫描报名二维码 | 扫描到响应时间 < 2s；状态显示 | 显示用户姓名/分组并标记 checkin_time，重复扫码提示已签到 | 夏晋 | 全通过 |
| Blocked user attempt | 验证惩罚期阻止报名 | 1) 模拟 blocked_until > now 用户 2) 点击报名 | 请求 403；提示封禁原因 | 报名被拒，中文/英文提示封禁截止时间 | 夏晋 | 全通过 |
| Analytics dashboard load | 验证运营端可用性 | 1) 登录 staff 2) 打开 analytics 总览 | 首屏加载 < 2s；图表正常 | 展示今日报名/签到等指标，无错误或空白组件 | 夏晋 | 全通过 |
| Performance guardrail | 确认核心接口性能 | 调用 `/api/events/search` & `/api/analytics/events/<eid>/overview` | 95th 延迟 < 1s（示例数据集）；无 5xx | 接口响应在阈值内，返回结构完整 | 夏晋 | 全通过 |